# Scenes
__________________________________

## Working with Scenes
____________________________

[**Scenes**](https://docs.descarteslabs.com/guides/scenes.html) is a tool in the Descartes Labs platform to search and access imagery over a wide variety of data sources. Below we show how to define an area of interest, search for imagery, download the image, and start doing analysis. First let's import the Descartes Labs library and define a place of interest.

You can run the following cells using `Shift-Enter`.

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
import descarteslabs as dl
import numpy as np

Our area of interest is [Folsom Lake](https://en.wikipedia.org/wiki/Folsom_Lake), a large resevoir in northern California. We can define an area of interest by specifying a [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) feature object.

In [ ]:
feature = {
    "type":"Feature",
    "geometry":{
    "type":"Polygon",
        "coordinates":[[
            [-121.18599701346828, 38.693120375759946],
            [-121.08821867639196, 38.693120375759946],
            [-121.08821867639196, 38.77282238770522],
            [-121.18599701346828, 38.77282238770522],
            [-121.18599701346828, 38.693120375759946]
        ]]
    },
    "properties": {}
} 
fc = {"type": "FeatureCollection", "features": [feature], "properties": {}}

In [ ]:
from ipyleaflet import Map, GeoJSON
m = Map(center=(38.733120, -121.1221), zoom=12)
m.add_layer(GeoJSON(data=fc))
m

If we want to find imagery over the region we can query for it using the [scenes.search](https://docs.descarteslabs.com/descarteslabs/scenes/docs/search.html) method. The search function will return a [SceneCollection](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scenecollection.html) object, a collection of metadata associated with images that match our query. It also returns a [GeoContext](https://docs.descarteslabs.com/descarteslabs/scenes/docs/geocontext.html) which has metadata about the region of interest, projections, and scale.

In [ ]:
scenes, ctx = dl.scenes.search(
    aoi=feature,
    products='sentinel-2:L1C',
    start_datetime='2019-07-31',
    end_datetime='2019-08-05'
)

In [ ]:
scenes

In [ ]:
ctx

Individual [scenes](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scene.html) in a SceneCollection can be accessed by indexing the collection just as if the data was a list. Inspecting a scene displays a detailed list of properties including information about the sensor, bands, and timestamps associated with the image. 

In [ ]:
scenes[0]

We can even get individual properties from a scene under its property field, which can be accessed like an attribute or a dictionary.

In [ ]:
scenes[0].properties.date
# or scenes[0].properties['date']

Although we have just queried for matching images, we have not downloaded anything yet. Getting the pixels from any given scene is as easy as calling the [ndarray](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scene.html#descarteslabs.scenes.scene.Scene.ndarray) method on a scene. The function returns an numpy array that is then ready to be used for further analysis. We need to assign a resolution to the GeoContext and choose which bands we want. 

In [ ]:
ctx = ctx.assign(resolution=10)

In [ ]:
a = scenes[0].ndarray(
    bands='red green blue',
    ctx=ctx
)

In [ ]:
a.shape

In [ ]:
%matplotlib inline
dl.scenes.display(a)

## Programmatic Data Access
___________

Scenes allows programatic access to imagery. That means we can easily specify the parameters of our images and change them with minimal work. For example, if we are interested in retrieving our imagery at a different resolutions, we can just change the matching parameter in the GeoContext. Resampling is controlled by the resampler keyword argument in the [ndarray](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scene.html#descarteslabs.scenes.scene.Scene.ndarray) method, and defaults to nearest neighbors. We might be interested in downsampling our scenes when we want to access imagery over large areas of interest and we don't need the full resolution. 

In [ ]:
import matplotlib.pyplot as plt
arrays = []
resolution = [20, 200, 500]
for res in resolution:
    # assignment yields a new geocontext with the modified property
    ctx = ctx.assign(resolution=res)
    a = scenes[0].ndarray(
        bands='red green blue',
        ctx=ctx,
        bands_axis=-1,
        scaling=[(0, 2500)] * 3
    )
    arrays.append(a)
    
fig, ax = plt.subplots(1, 3, figsize=(18, 10))
for i in range(3):
    ax[i].imshow(arrays[i])
    ax[i].set_title('Resolution: {}m'.format(resolution[i]))

We can also pull our imagery with different band combinations. We can find the list of bands under the `properties.bands` field of each scene. In this case we can see that the Sentinel-2 satellite has a wide variety of visual, infrared, and thermal bands. 

In [ ]:
# print available bands for this scene:
scenes[0].properties.bands.keys()

Let's visualize a few band combinations. The most familiar is the visual red, green, blue bands. Near infrared, *nir*, is often used to identify vegetation. Finally the thermal bands *swir1* and *swir2* are often used to identify hotspots such as wildfires or flares. 

In [ ]:
arrays = []
ctx = ctx.assign(resolution=20)
bands = ['red green blue', 'nir red green', 'swir1 swir2 nir']
for b in bands:
    a = scenes[0].ndarray(
        bands=b,
        ctx=ctx,
        bands_axis=-1,
        scaling=[(0, 2500)] * 3
    )
    arrays.append(a)
    
fig, ax = plt.subplots(1, 3, figsize=(18, 10))
for i in range(3):
    ax[i].set_title('Bands: {}'.format(bands[i]))
    ax[i].imshow(arrays[i])

Finally, let's look at the same scene, but over a couple of different sensors. Each satellite has different types of sensors for different types of analysis.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 10))
products = ['sentinel-2:L1C', 'landsat:LC08:01:T1:TOAR', 'sentinel-1:GRD']
bands = ['red green blue', 'swir1 swir2 nir', 'vv vh vv']
scaling = [[(0, 2500)] * 3, [(0, 5000)] * 3, None]
for i in range(3):
    s, ctx = dl.scenes.search(feature, products[i], start_datetime='2018-06-01', end_datetime='2018-06-30')
    ctx = ctx.assign(resolution=10)
    ax[i].imshow(s.mosaic(bands=bands[i], ctx=ctx, bands_axis=-1, scaling=scaling[i]))
    ax[i].set_title(products[i])

One of the strengths of Scenes is that it is very easy to retrieve a wide variety of spatial data from dozens of sources.

## Wrap Up
__________________

In this notebook we have implemented a simple water masking algorithm. In the process we have:

* Used [scenes.search](https://docs.descarteslabs.com/descarteslabs/scenes/docs/search.html) to find matching images over a region of interest.
* Used [scenes.ndarray](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scene.html#descarteslabs.scenes.scene.Scene.ndarray) to get the pixels for any given scene.
* Access scenes with a wide variety of parameters on the fly.

## Resources
________________________

[1] [Sentinel-2 product](https://catalog.descarteslabs.com/?/product/sentinel-2:L1C)

[2] [Scenes API Documentation](https://docs.descarteslabs.com/descarteslabs/scenes/readme.html)
